<a href="https://colab.research.google.com/github/GabrielCachoa/Sat-Heuristics/blob/main/SAT_Heuristics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#SAT - Heuristics

# Needed Packages

In [ ]:
import numpy as np
from numpy import random
import matplotlib.pyplot as plt
import timeit
import pandas 
import csv
import time
import glob
from os.path import isfile


# Load Instances

Input: path of a cnf DIIMACS sat instance (string)

Output: (p,P) where p is the line of the problem and P is a numpy array that represents the instance as a matrix

p = [ number of variables, number of clauses ] (list)

P = [[var1, var2, var3], ... , [vari, varj, vark]] (numpy array)

In [ ]:
def load(fn):
  f = open(fn, 'r')
  data = f.read()
  f.close()
  clauses = []
  lines = data.split("\n")
  p=[]

  for line in lines:
    if len(line) == 0 or line[0] in ['c', '%', '0']:
      continue

    if line[0]=='p':
      problemline=line.split()
      p.extend([int(problemline[2]),int(problemline[3])])
    else:
      clause = [int(x) for x in line.split()[0:-1]]
      clauses.append([x for x in clause])


  P=np.array(clauses)

  return (p,P)


# Objective Function

**Input**: (P,s) where P is a numpy array (output of load) and s is solution for P. That means: s is a numpy array of 0's and 1's of lenght the number of variables in P. 

**Output**: n (int) where n is the number of satisfied clauses in P with the variables assignament given by s. 

In [ ]:
def f(P,s):
  (n,m)=P.shape
  values=np.zeros(n)
  Q=np.copy(P)
  for i in range(n):
    for j in range(m):
      if s[np.abs(Q[i,j])-1]==0:
        Q[i,j]=Q[i,j]*(-1)
    values[i]=np.sign(np.max(Q[i,:]))
  values[values==-1]=0
  return sum(values)

#Neighborhood Structures

We defined two neigborhood structures for a solution. One deterministic (Neighborhood) and one non-deterministic (Neighborhood2).



###Neighborhood

**Input**: (p,s) where p is the line problem and s is a solution for the problem P, meaning is a numpy array of numpy array of 0's and 1's of length p[0].

**Output**: V (numpy array) of shape (p[0]+1,p[0]). Each row of this numpy array is neighbor of s. s is the first row of V. In this neighboorhood, a numpy array of 0's and 1's is a neighbor of s if they differ by just one entry.


###Neighborhood2

**Input**: (p,s,n,c) where p and s are as in Neighborhood, n (int), and c (int). 

**Output**: V (numpy array) of shape (n+1,p[0]). As in Neighborhood, each row is a neighbor of s. Here we consider a neighbor of s a numpy array of 0's and 1's such that it differ of s by at most c entries. To construct the neighbors of s we randomly change entries in s. 


In [ ]:
def Neighborhood(p,s):
  m=p[0]
  V=np.zeros((m+1,m))
  V[0,:]=s
  for i in range(1,m+1):
    V[i,:] = s
    V[i,i-1] = np.abs(1-V[i,i-1])
  return V

In [ ]:
def Neighborhood2(p,s,n,c):
  m=p[0]
  V=np.zeros((n+1,m))
  V[0,:]=s
  random.seed(seed)
  for i in range(1,n+1):
    V[i,:] = s
    for j in range(c):
      c1=random.randint(0,m)
      V[i,c1]=1-V[i,c1]
  return V

# Local Search

**Input**: (P,V) where P (numpy array) output of load and V (numpy array) is a numpy array in which each row is a solution for P

**Output**: (f_star,s_star) where f_star (int) the maximum number of satisfied clauses in P by a row of V, s_star (numpy array) is such row. 


In [ ]:
def LocalSearch(P,V):
  v=[]
  for i in range(V.shape[0]):
    v.append(f(P,V[i,:]))
  f_star=max(v)
  s_star=V[v.index(max(v))]
  return (f_star,s_star)

#Hill Climbing

**Input**: (P,p,s,TIME) where (P,p) is an output of load, s (numpy array) is a solution for P and TIME (float) is the duration of time the heuristic will run.

**Output**: (f_star, s_star) f_star is the best value in the objective function found by Hill Climbing heuristic and s_star is the solution that gave that value. 


In [ ]:
def HillClimbing(P,p,s,TIME):
  start_time = time.time()
  now_time = time.time()

  s_star = np.copy(s)
  f_star = f(P,s_star)

  while now_time - start_time < TIME and f_star < p[1]:
    (f_star,s_star) = LocalSearch(P,Neighborhood(p,s_star))
    s = np.copy(s_star)
    now_time = time.time()
    
  return (f_star,s_star)

# Tabu Search

Here we define an auxiliary function: CleanTheNeighborhood. This function take the tabu list and a neigborhood and ereases every element from the tabu list that appears in the neighboorhood.


###CleanTheNeighborhood

**Input**: (V,TabuList) where V is a numpy array in which each row is an array of 0's and 1's. TabuList (list) is a list in which each element is a numpy array of 0's and 1's with lenght the widht of V.

**Output**: CleanV (numpy array) is the entry V but each row that appeared in TabuList was ereased. 


###TabuSearch
**Input**: (P,p,s,TIME) where (P,p) is an output of load, s (numpy array) is a solution for P and TIME (float) is the duration of time the heuristic will run.

**Output**: (f_star, s_star) f_star is the best value in the objective function found by Tabu Search heuristic and s_star is the solution that gave that value. 

In [ ]:
def CleanTheNeighborhood(V,TabuList):

  CleanV = np.copy(V)

  EraseList = []

  for i in range(V.shape[0]):
    for j in range(len(TabuList)):
      if np.array_equal(V[i,:],TabuList[j]):
        EraseList.append(i)
        
  
  for i in range(len(EraseList)):
    if CleanV.shape[0] != 0:
      EraseList[i] = EraseList[i]-i
      CleanV = np.delete(CleanV,EraseList[i],0)
    else:
      break

  return CleanV

In [ ]:
def TabuSearch(P,p,s,TIME):
  start_time=time.time()
  now_time=time.time()

  TabuList=[]
  s_star = np.copy(s)
  f_star = f(P,s) 

  while now_time - start_time < TIME and f_star < p[1]:

    V = Neighborhood(p,s_star)
    V = CleanTheNeighborhood(V,TabuList)

    if V.shape[0]!=0:

      (f_s,s) = LocalSearch(P,V)
      TabuList.append(s)

      if f_s > f_star:
        s_star = np.copy(s)
        f_star = f_s
        
      now_time = time.time()

    else:
      now_time = time.time()
  
  return (f_star,s_star)

# Simulated Annealing

**Input**: (P,p,s,t,alpha,TIME) where (P,p) is an output of load, s (numpy array) is a solution for P and TIME (float) is the duration of time the heuristic will run.

t (float) is the initial value for temperature

alpha (float) is the value that determines the cooling schedule



**Output**: (f_star, s_star) f_star is the best value in the objective function found by Simulated Annealing heuristic and s_star is the solution that gave that value. 

In [ ]:
def SimulatedAnnealing(P,p,s,t,alpha,TIME):

  s_star = np.copy(s)
  f_star = f(P,s_star)
  
  start_time = time.time()
  now_time = time.time()

  while now_time - start_time < TIME and f_star < p[1]:
    V = Neighborhood(p,s_star)
    (f_s,s) = LocalSearch(P,V)
    if f_s >= f_star:
      f_star = f_s
      s_star = np.copy(s)

    elif np.exp((f_star - f_star)/t) > random.uniform():
      f_star = f_s
      s_star = np.copy(s)


    t = t* alpha
    now_time=time.time()
  

  return (f_star,s_star)

# GRASP

Here we define 5 auxiliary functions: greedy_f, Candidates, RCL, SolutionFormat,and GreedyRandomizedConstruction and the heuristic GRASP.

S is going to be a *partial solution* for a problem P. S is a list of integers bounded by the interval [-p[0],p[0]]. 


###greedy_f

**Input**: (P,S) where P is an output of load and S is as described.

**Output**: values (int) the number of clauses satisfied by the partial solution S in the problem P.


###Candidates

**Input**: (p,S) p is an output of load and S is a patial solution.

**Output**: L (list) a list of all the posible candidates (integers) that does not appear in the partial solution S.


###RCL

**Input**: (P,L,S,alpha) where P is the output of load, L is the output of Candidates, S is a partial solution (the input in candidates), alpha (float) is the degree of randomness of the procedure.

**Output**: CandidatesRestrictedList (list) is a sublist of L that satisfy the condition restricted candidate list procedure.

\
###SolutionFormat

**Input**: S partial solution of lenght p[0] (the number of variables).

**Output**: s numpy array of 0's and 1's that correspond to the partial solution S. 


###GreedyRandomizedConstruction

**Input**: (P,p,alpha) where (P,p) is an output of load and alpha (float) is the degree of randomness of the procedure.

**Output**: s_star a solution constructed by the GreedyRandomizedConstruction procedure.


###GRASP

**Input**: (P,p,alpha,TIME) where (P,p) is an output of load, alpha (float) is the degree of randomness of the procedure and TIME (float) is the duration of time the heuristic will run.


**Output**: (f_star, s_star) f_star is the best value in the objective function found by GRASP heuristic and s_star is the solution that gave that value. 

In [ ]:
def greedy_f(P,S):
  (n,m)=P.shape
  v=np.zeros(n)
  Q=np.copy(P)
  for i in range(n):
    for j in range(m):
      if Q[i,j] in S:
        v[i]=1
        break
  values = sum(v)
  return values

def Candidates(p,S):
  id_plus = lambda x : x
  id_minus = lambda x : -x

  Range = list(range(1,p[0]+1))

  for i in range(len(S)):
    Range.remove(np.abs(S[i]))

  L = [i(x) for x in Range for i in (id_plus, id_minus)]
  
  return L


def RCL(P,L,S,alpha):

  S_copy = np.copy(S)
  GreedyValue = [greedy_f(P,S+[L[i]]) for i in range(len(L))]

  Max = np.max(GreedyValue)
  Min = np.min(GreedyValue)
  value = Max - alpha * (Max - Min)

  CandidatesRestrictedList = []
  for i in range(len(L)):
    if GreedyValue[i] > value:
      CandidatesRestrictedList.append(L[i])

  return CandidatesRestrictedList


def SolutionFormat(p,S):

  s = np.array([1 if i+1 in S else 0 for i in range(p[0])])
  
  return s

def GreedyRandomizedConstruction(P,p,alpha):
  S = []
  L = Candidates(p,S)
  GreedyValue = [greedy_f(P,[L[i]]) for i in range(len(L))]

  while len(L) != 0:
    
    RCL_list = RCL(P,L,S,alpha)

    if len(RCL_list) != 0:
      index = random.randint(0,len(RCL_list))
      Choosen = RCL_list[index]
      S.append(Choosen)
      L = Candidates(p,S)

    else:
      GreedyValue = [greedy_f(P,[L[i]]) for i in range(len(L))]
      Choosen = L[np.argmax(GreedyValue)]
      S.append(Choosen)
      L = Candidates(p,S)

  s_star = SolutionFormat(p,S)

  return s_star

def GRASP(P,p,alpha,TIME):
  start_time = time.time()
  now_time = time.time()

  f_star = 0

  random.seed(seed)

  while now_time - start_time < TIME:

    s0 = GreedyRandomizedConstruction(P,p,alpha)

    V = Neighborhood(p,s0,n,c,seed)
    (f_s1,s1) = LocalSearch(P,V)

    if f_s1 > f_star:
      s_star = np.copy(s1)
      f_star = f_s1

    now_time = time.time()
  
  return (f_star,s_star)

# Variable Neighborhood Search

Here we defined an auxiliary function: Shaking. And we are going to be using Neigborhood2 to define the neigborhood structures. 


### Shaking

**Input**: (P,p,s_star,k,Sizes) where (P,p) is the output of the function load, s_star (numpy array) is a solution for P, k (int) represents the kth neighborhood structure and Sizes is a list in which each element is a list of two elements [n,c] that correspond to the input in the function Neighborhood2.

**Output**: s_0 (numpy array) is a randomly choosen solution in the kth neighborhood structure of the given solution s_star.


### VariableNeighborhoodSearch

**Input**: (P,p,s,Sizes,TIME) where (P,p) is the output of the function load, Sizes is a as lately described and TIME (float) is the duration of time the heuristic will run.


Output: (f_star, s_star) f_star is the best value in the objective function found by VariableNeighborhoodSearch heuristic and s_star is the solution that gave that value. 

In [ ]:
def Shaking(P,p,s_star,k,Sizes):

  V = Neighborhood2(p,s_star,Sizes[k][0],Sizes[k][1])
  i = random.randint(0,V.shape[0])

  s_0 = V[i,:]

  return s_0

def VariableNeighborhoodSearch(P,p,s,Sizes,TIME):
  start_time = time.time()
  now_time = time.time()
  
  
  random.seed(seed)

  s_star = InitialSolution(p)
  f_star = f(P,s_star)

  k_max = len(Sizes)

  k_counter =np.zeros(k_max)

  T = [[s_star]]

  while (now_time - start_time < TIME) and f_star < p[1]:
    k=0
    while k < k_max:

      if np.array_equal(T[-1][0],s_star):
        T[-1].append(k)
      else:
        T.append([s_star,0])

      s_0 = Shaking(P,p,s_star,k,Sizes)
      (f_s1,s1) = LocalSearch(P,NeighborhoodVNS(p,s_0,Sizes[k][0],Sizes[k][1]))

      k_counter[k] = k_counter[k]+1

      if f_s1 > f_star:
        s_star = np.copy(s1)
        f_star = f_s1

        k = 0
      else:
        
        k = k + 1


    now_time = time.time()

  return (f_star,s_star)

# Genetic Algorithms

Here we define an auxiliar function for each stage of theb heuristic. 

Let us stablish that (P,p) is the output of th


###Initialization

**Input:** (P,p,size) where size (integer) is the size of the population

**Output:** Population (numpy array) in which each row is a solution of P.


###Selection

**Input:** (P,p,Population) where Population (numpy array) satisfies that each row is a solution of P.

**Output:** MatingPool (list) is a list of pairs of row indexes of the best solutions in Population regarding the objective function.


###Recombination

**Input:** (P,p,MatingPool) where MatingPool is the output of Selection.

**Output:** Offspring (list) is the list of the resulting mating of each pair of elements in MatingPool


###Mutation

**Input:** (P,p,Offspring) where Offspring is a list of solutions for P.

**Output:** Offspring (list) is the entry but with probablity 20% each solution was changed in one randomly choosen entry.


###Replacement(P,p,Population,Offspring)

**Input:** (P,p,Population,Offspring) where Offspring (list) is a list of posible solutions P and Population (numpy array) is an array in wich each row is a solution for P.

**Output:** Population (numpy array) is a numpy array in wich the worst solutions regarding the objective function where replaced by the Offspring


###Genetic Algorithm(P,p,size,TIME)

 (P,p,s,size,TIME) where (P,p) is the output of the function load, size (int) is the size of the population and TIME (float) is the duration of time the heuristic will run.


Output: (f_star, s_star) f_star is the best value in the objective function found by GeneticAlgorithm heuristic and s_star is the solution that gave that value. 

In [ ]:
def Initialization(P,p,size):

  Population = []
  for i in range(size):
    Population.append(random.randint(0,2,p[0]))
  Population=np.array(Population)

  return Population

def Selection(P,p,Population,size):

  a=np.arange(Population.shape[0])
  MatingPool=[]
    
  Values = [f(P,Population[i,:]) for i in range(Population.shape[0])]
  sum = np.sum(Values)
  P = [value/sum for value in Values]

  for i in range(int(size/5)):
    MatingPool.append(random.choice(a,2,p))

  return MatingPool

def Recombination(P,p,MatingPool,Population):
  Offspring=[]

  for pair in MatingPool:
    n=random.randint(p[0])
    son=np.append(Population[pair[0],0:n],Population[pair[1],n:p[0]])
    Offspring.append(son)
 return Offspring

def Mutation(P,p,Offspring):

  M=random.uniform(0,1,len(Offspring))
  M[M > 0.2]=0
  M[M != 0]=1

  for i in range(len(Offspring)):
    if M[i]==1:
      l=Offspring[i].shape[0]
      n=random.randint(l)
      Offspring[i][n]=1-Offspring[i][n]

  return Offspring

def Replacement(P,p,Population,Offspring,size):
  new_population=[]
  for i in range(size):
    new_population.append(np.append(Population[i,:],f(P,Population[i,:])))

  new_population=np.array(new_population)
  new_population = new_population[np.argsort(new_population[:, -1])]
  new_population=np.delete(new_population,-1,1)
  new_population=np.delete(new_population,range(int(size/5)),0)

  for son in Offspring:
    new_population=tuple(new_population)
    new_population=np.vstack((new_population,son))
    
  new_population=np.array(new_population)
  return Population

def GeneticAlgorithm(P,p,size,TIME):
  
  start_time = time.time()
  now_time = time.time()

  random.seed(seed)

  Population = Initialization(P,p,size)

  f_star = 0
  while now_time - start_time < TIME and f_star < p[1]:


    MatingPool = Selection(P,p,Population,size)

    Offspring = Recombination(P,p,MatingPool,Population)

    Offspring = Mutation(P,p,Offspring)

    Population = Replacement(P,p,Population,Offspring,size)

    now_time = time.time()
  
    (f_star,s_star) = LocalSearch(P,Population)

  return (f_star,s_star) 


# Optimización por colonias de hormigas

PheromonesInitialization(p)

ProbabilitiesMatrix(p,Pheromones)

Candidates(p,S)

Choose(L,Probabilities)



In [ ]:
def PheromonesInitialization(p):

  Pheromones = np.full((2*p[0]+1,2*p[0]+1),0.5)

  L = Candidates(p,[])

  for i in range(2*p[0]+1):
    for j in range(2*p[0]+1):
      
      if j > 0:
        Pheromones[0,j] = L[j-1]

      if i == j:
        Pheromones[i,j] = 0
      elif (i%2) == 0 and j == i-1:
        Pheromones[i,j] = 0
      elif (i%2) != 0 and j == i+1:
        Pheromones[i,j] = 0
    
    if i > 0:
      Pheromones[i,0] = L[i-1]


  return Pheromones

def ProbabilitiesInitialization(p):
  Probabilities = np.full((2*p[0]+1,2*p[0]+1),1 / (p[0]-2))

  L = Candidates(p,[])

  for i in range(2*p[0]+1):
    for j in range(2*p[0]+1):
      
      if j > 0:
        Probabilities[0,j] = L[j-1]

      if i == j:
        Probabilities[i,j] = 0
      elif (i%2) == 0 and j == i-1:
        Probabilities[i,j] = 0
      elif (i%2) != 0 and j == i+1:
        Probabilities[i,j] = 0
    
    if i > 0:
      Probabilities[i,0] = L[i-1]
  return Probabilities

def Choose(p,S,L,Probabilities):
 
  ALLCandidates = Probabilities[:,0]
  ActualPosition = np.where(ALLCandidates == S[-1])[0]
  RowOfProbabilities = Probabilities[ActualPosition,:]
  
  ProbabilitiesL = []

  for i in range(len(L)):
    index = np.where(ALLCandidates == L[i])[0][0]
    ProbabilitiesL.append(RowOfProbabilities[0,index])
  
  Sum = np.sum(ProbabilitiesL)
  
  NormalizedProbabilities = [p/Sum for p in ProbabilitiesL]

  Choosen = random.choice(L,p = NormalizedProbabilities, size = 1)[0]

  return Choosen

def PheromonesIntensification(Pheromones,BestWalk,delta):
  ALLCandidates = Pheromones[:,0]
  for i in range(len(BestWalk)-1):
    index0 = np.where(ALLCandidates == BestWalk[i])[0][0]
    index1 = np.where(ALLCandidates == BestWalk[i+1])[0][0]

    Pheromones[index0,index1] = Pheromones[index0,index1] + delta
    
  return Pheromones

def ProbabilitiesMatrix(Probabilities,Pheromones):
  for i in range(1,Probabilities.shape[0]):
    Sum = np.sum(Pheromones[i,1:])
    for j in range(1,Probabilities.shape[1]):
      Probabilities[i,j] = Pheromones[i,j] / Sum
  
  return Probabilities

def AntColonyOptimization(P,p,k,rho,delta,seed,TIME):

  start_time = time.time()
  now_time = time.time()

  random.seed(seed)

  Pheromones = PheromonesInitialization(p)

  Probabilities = ProbabilitiesInitialization(p)

  random.seed(seed)

  f_star = 0

  while now_time - start_time < TIME and f_star < p[1]:

    Solutions = []
    Walks = []
    for i in range(5):
      S = []
      L = Candidates(p,S)
      
      S.append(L[random.randint(len(L))])

      while len(L) != 0:
        Choosen = Choose(p,S,L,Probabilities)
        S.append(Choosen)
        L = Candidates(p,S)

      s = SolutionFormat(p,S)
      Solutions.append(s)
      Walks.append(S)

    Values = [f(P,s) for s in Solutions]

    BestWalk = Walks[Values.index(max(Values))]

    s_star = Solutions[Values.index(max(Values))]
    f_star = max(Values)

    for i in range(1,Pheromones.shape[0]):
      for j in range(1,Pheromones.shape[1]):
        Pheromones[i,j] = Pheromones[i,j] * (1-rho) 


    Pheromones = PheromonesIntensification(Pheromones,BestWalk,delta)

    Probabilities = ProbabilitiesMatrix(Probabilities,Pheromones)

    now_time = time.time()


  return (f_star,s_star)
